In [8]:
import pandas as pd
import numpy as np
import glob
import re
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from kmodes import kmodes,kprototypes
from datetime import datetime, timedelta

%reload_ext autoreload
%autoreload 2

### To-Do: 
#### 1. Create different csvs of hourly health costs for each discount rate.

#### Read in EPA AMPD emissions data for LADWP gas plants.

In [9]:
#Read in EPA AMPD emissions data from power plants owned or operated by LADWP.
ampd_ladwp = pd.read_csv('data/grid_emissions/epa_ampd_ladwp_hourly_emissions_2019.csv')
ampd_ladwp = ampd_ladwp.fillna(0)

In [10]:
#Convert lbs to tons.
ampd_ladwp['so2_ton']=ampd_ladwp['so2_lb']/2000
ampd_ladwp['nox_ton']=ampd_ladwp['nox_lb']/2000

In [11]:
ampd_ladwp['date'] = pd.to_datetime(ampd_ladwp['date'])

In [12]:
#Sum emissions across plants in each hour.

#ampd_ladwp_grouped = ampd_ladwp.groupby(['date', 'hour','year']).sum().reset_index()


# ladwp_hourly_emissions_gas = ampd_ladwp_grouped.sort_values(by=['date','hour'])
# ladwp_hourly_emissions_gas.rename(columns={'load_MW':'load_MW_gas','so2_lb': 'so2_lb_gas','nox_lb':'nox_lb_gas',
#                                            'co2_short_tons':'co2_short_tons_gas'},inplace=True)

#### Read in EPA AMPD emissions data for IPP coal plant.

In [13]:
#Read in EPA AMPD emissions data from Intermountain Power Plant (coal plant supplying LADWP).
ampd_ipp = pd.read_csv('data/grid_emissions/epa_ampd_intermountain_hourly_emissions_2019.csv')
ampd_ipp = ampd_ipp.fillna(0)
ampd_ipp['date'] = pd.to_datetime(ampd_ipp['date'])

In [14]:
#Convert lbs to tons.
ampd_ipp['so2_ton']=ampd_ipp['so2_lb']/2000
ampd_ipp['nox_ton']=ampd_ipp['nox_lb']/2000

#### Loop through folder of EIA LADWP hourly gen data by energy source and concat monthly datasets.

In [15]:
#Loop through folder of files with EIA LADWP hourly net generation data by energy source, concatenate into one file.
path = 'data/grid_emissions/LADWP_hourly_gen_by_source'
filenames = glob.glob(path + "/*.csv")

dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename))

# concatenate into one dataframe
hourly_gen_by_source = pd.concat(dfs, ignore_index=True)
hourly_gen_by_source.columns = hourly_gen_by_source.columns.str.replace('\s+', '_') 

#### Reformat EIA hourly generation by energy source data to align with datetime of EPA data.

In [16]:
#Split date and hour (ending).
hourly_gen_by_source = hourly_gen_by_source.join(hourly_gen_by_source['Timestamp_(Hour_Ending)'].str.split(r' ',1,expand=True).rename(columns={0:'date', 1:'hour_ending'}))

#Drop duplicate datetimes.
hourly_gen_by_source.drop_duplicates(ignore_index=True, inplace=True)

#Split hour (ending) and am/pm.
hourly_gen_by_source = hourly_gen_by_source.join(hourly_gen_by_source['hour_ending'].str.split(r' ',1,expand=True).rename(columns={0:'hour_ends', 1:'am/pm'}))

hourly_gen_by_source['am/pm'] = hourly_gen_by_source['am/pm'].str[:-4]
hourly_gen_by_source['date'] = pd.to_datetime(hourly_gen_by_source['date'])
hourly_gen_by_source['hour_ends'] = pd.to_numeric(hourly_gen_by_source['hour_ends'])

#### Fill in rows with nan values with total net generation and coal generation from previous day on same hour.

In [17]:
for index, row in hourly_gen_by_source[hourly_gen_by_source['Total_Generation_(MWh)'].isna()].iterrows():
    date = hourly_gen_by_source.loc[index,'date']
    date_previous = date-timedelta(days=1)
    time = hourly_gen_by_source.loc[index,'hour_ends']
    am_pm = hourly_gen_by_source.loc[index,'am/pm']
    
    previous_day_inds = hourly_gen_by_source['date'] == date_previous
    time_inds = hourly_gen_by_source['hour_ends'] == time
    am_pm_inds = hourly_gen_by_source['am/pm'] == am_pm
    previous_gen = hourly_gen_by_source[previous_day_inds & time_inds & am_pm_inds]['Total_Generation_(MWh)'].item()
    previous_coal = hourly_gen_by_source[previous_day_inds & time_inds & am_pm_inds]['Coal_Generation_(MWh)'].item()
    if previous_gen == 0:
        print(index)
    if previous_coal == 0:
        print(index)
    
    hourly_gen_by_source.loc[index,'Total_Generation_(MWh)'] = previous_gen
    hourly_gen_by_source.loc[index,'Coal_Generation_(MWh)'] = previous_coal
    
for index, row in hourly_gen_by_source[hourly_gen_by_source['Coal_Generation_(MWh)']<0].iterrows():
    hourly_gen_by_source.loc[index,'Coal_Generation_(MWh)']=0

In [18]:
pm_inds = hourly_gen_by_source['am/pm']=='p.m.'
am_inds = hourly_gen_by_source['am/pm']=='a.m.'

hourly_gen_by_source_pm = hourly_gen_by_source[pm_inds].reset_index()
for row in hourly_gen_by_source_pm.index:
    if hourly_gen_by_source_pm.loc[row,'hour_ends']<12:
        hourly_gen_by_source_pm.loc[row,'hour_ends']=hourly_gen_by_source_pm.loc[row,'hour_ends']+12

hourly_gen_by_source_am = hourly_gen_by_source[am_inds].reset_index()
for row in hourly_gen_by_source_am.index:
    if hourly_gen_by_source_am.loc[row,'hour_ends']== 12:
        hourly_gen_by_source_am.loc[row,'date']= hourly_gen_by_source_am.loc[row,'date']-timedelta(days=1)
        hourly_gen_by_source_am.loc[row,'hour_ends'] = 24
        
hourly_gen_reformatted = pd.concat([hourly_gen_by_source_am, hourly_gen_by_source_pm], ignore_index=True)

hourly_gen_reformatted['hour'] = (hourly_gen_reformatted['hour_ends'])-1
hourly_gen_reformatted.drop(columns=['index','hour_ending','hour_ends','am/pm','Timestamp_(Hour_Ending)'], inplace=True)
hourly_gen_reformatted = hourly_gen_reformatted.sort_values(by=['date','hour'])
hourly_gen_by_source_reformatted = hourly_gen_reformatted[hourly_gen_reformatted.date >= '2019-01-01']

#### Calculate fraction of hourly coal and natural gas emissions attributed to LADWP. Code currently assumes EIA source is correct, scales coal emissions from EPA AMPD dataset up to coal generation in EIA dataset.

In [19]:
eia_hourly_coal_gas = hourly_gen_by_source_reformatted[['date','hour','Coal_Generation_(MWh)','Natural_gas_Generation_(MWh)','Total_Generation_(MWh)']]

In [20]:
merged = eia_hourly_coal_gas.merge(ampd_ipp, on=['date','hour'], how='inner')

In [21]:
merged['ladwp_fraction_of_coal_gen'] = merged['Coal_Generation_(MWh)']/merged['load_mw']

In [22]:
merged['ladwp_coal_so2_ton'] = merged['so2_ton']* merged['ladwp_fraction_of_coal_gen']
merged['ladwp_coal_nox_ton'] = merged['nox_ton']* merged['ladwp_fraction_of_coal_gen']
merged['ladwp_coal_co2_short_tons'] = merged['co2_short_tons']* merged['ladwp_fraction_of_coal_gen']

In [23]:
merged.rename(columns={'Coal_Generation_(MWh)':'load_MW'},inplace=True)

In [24]:
coal_emissions = merged[['date','hour','facility','load_MW','Natural_gas_Generation_(MWh)','Total_Generation_(MWh)','ladwp_coal_so2_ton','ladwp_coal_nox_ton','ladwp_coal_co2_short_tons']]

#### Merge hourly LADWP coal plant emissions with hourly LADWP gas plant emissions and divide by total hourly generation.

In [25]:
coal_emissions.rename(columns={'ladwp_coal_so2_ton':'so2_ton','ladwp_coal_nox_ton':'nox_ton','ladwp_coal_co2_short_tons':'co2_short_tons'}, inplace=True)

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4295: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [26]:
summed_emissions = pd.concat([coal_emissions,ampd_ladwp],ignore_index = True)

In [27]:
summed_emissions = summed_emissions[['date','hour','facility','load_MW','so2_ton','nox_ton','co2_short_tons']]

In [28]:
grouped = summed_emissions.groupby(['facility'])

### Read in health costs

In [30]:
health_costs = pd.read_csv('data/pollutant_health_impacts/COBRA_LADWPplants_healthCosts.csv')

In [31]:
rejoined = pd.DataFrame()
for name, group in grouped:
    power_plant = name.split(' ', 1)[0]

    health_costs_filtered = health_costs[health_costs['power_plant']==power_plant]
    so2_inds = health_costs_filtered['pollutant']=='SO2'
    nox_inds = health_costs_filtered['pollutant']=='NOx'
    pm25_inds = health_costs_filtered['pollutant']=='PM2.5'
    discount_rate_3pct_inds = health_costs_filtered['discount_rate']==0.03
    discount_rate_6pct_inds = health_costs_filtered['discount_rate']==0.06
    
    group['so2_cost_per_ton_LOW_3pct'] = health_costs_filtered[so2_inds & discount_rate_3pct_inds]['US_LOW_annual ($/ton)'].item()
    group['nox_cost_per_ton_LOW_3pct'] = health_costs_filtered[nox_inds & discount_rate_3pct_inds]['US_LOW_annual ($/ton)'].item()
    group['pm25_cost_per_ton_LOW_3pct'] = health_costs_filtered[pm25_inds & discount_rate_3pct_inds]['US_LOW_annual ($/ton)'].item()
    group['so2_cost_per_ton_HIGH_3pct'] = health_costs_filtered[so2_inds & discount_rate_3pct_inds]['US_HIGH_annual ($/ton)'].item()
    group['nox_cost_per_ton_HIGH_3pct'] = health_costs_filtered[nox_inds & discount_rate_3pct_inds]['US_HIGH_annual ($/ton)'].item()
    group['pm25_cost_per_ton_HIGH_3pct'] = health_costs_filtered[pm25_inds & discount_rate_3pct_inds]['US_HIGH_annual ($/ton)'].item()
    
    group['so2_cost_per_ton_LOW_6pct'] = health_costs_filtered[so2_inds & discount_rate_6pct_inds]['US_LOW_annual ($/ton)'].item()
    group['nox_cost_per_ton_LOW_6pct'] = health_costs_filtered[nox_inds & discount_rate_6pct_inds]['US_LOW_annual ($/ton)'].item()
    group['so2_cost_per_ton_HIGH_6pct'] = health_costs_filtered[so2_inds & discount_rate_6pct_inds]['US_HIGH_annual ($/ton)'].item()
    group['nox_cost_per_ton_HIGH_6pct'] = health_costs_filtered[nox_inds & discount_rate_6pct_inds]['US_HIGH_annual ($/ton)'].item()
    group['pm25_cost_per_ton_LOW_6pct'] = health_costs_filtered[pm25_inds & discount_rate_6pct_inds]['US_LOW_annual ($/ton)'].item()
    group['pm25_cost_per_ton_HIGH_6pct'] = health_costs_filtered[pm25_inds & discount_rate_6pct_inds]['US_HIGH_annual ($/ton)'].item()
    
    rejoined = pd.concat([rejoined, group],ignore_index=True)

In [32]:
rejoined

,date,hour,facility,load_MW,so2_ton,nox_ton,co2_short_tons,so2_cost_per_ton_LOW_3pct,nox_cost_per_ton_LOW_3pct,pm25_cost_per_ton_LOW_3pct,so2_cost_per_ton_HIGH_3pct,nox_cost_per_ton_HIGH_3pct,pm25_cost_per_ton_HIGH_3pct,so2_cost_per_ton_LOW_6pct,nox_cost_per_ton_LOW_6pct,so2_cost_per_ton_HIGH_6pct,nox_cost_per_ton_HIGH_6pct,pm25_cost_per_ton_LOW_6pct,pm25_cost_per_ton_HIGH_6pct
0,2019-01-01,0,Apex Generating Station,0.0,0.000000,0.000000,0.0,-20763,-11100,-96042,-46776,-25006,-216435,-6018,-4028,-13573,-9084,-87975,-198106
1,2019-01-01,1,Apex Generating Station,0.0,0.000000,0.000000,0.0,-20763,-11100,-96042,-46776,-25006,-216435,-6018,-4028,-13573,-9084,-87975,-198106
2,2019-01-01,2,Apex Generating Station,0.0,0.000000,0.000000,0.0,-20763,-11100,-96042,-46776,-25006,-216435,-6018,-4028,-13573,-9084,-87975,-198106
3,2019-01-01,3,Apex Generating Station,0.0,0.000000,0.000000,0.0,-20763,-11100,-96042,-46776,-25006,-216435,-6018,-4028,-13573,-9084,-87975,-198106
4,2019-01-01,4,Apex Generating Station,0.0,0.000000,0.000000,0.0,-20763,-11100,-96042,-46776,-25006,-216435,-6018,-4028,-13573,-9084,-87975,-198106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52555,2019-12-31,19,Valley Generating Station,501.0,0.001034,0.012067,201.1,-420843,-86957,-892079,-944650,-195177,-2002088,-385541,-79662,-864652,-178648,-817246,-1832532
52556,2019-12-31,20,Valley Generating Station,507.0,0.001045,0.012192,203.2,-420843,-86957,-892079,-944650,-195177,-2002088,-385541,-79662,-864652,-178648,-817246,-1832532
52557,2019-12-31,21,Valley Generating Station,508.0,0.001050,0.012242,204.0,-420843,-86957,-892079,-944650,-195177,-2002088,-385541,-79662,-864652,-178648,-817246,-1832532
52558,2019-12-31,22,Valley Generating Station,509.0,0.001051,0.012257,204.3,-420843,-86957,-892079,-944650,-195177,-2002088,-385541,-79662,-864652,-178648,-817246,-1832532


### When get pm2.5 emissions by power plant, change following lines to incorporate pm2.5 health costs.

In [33]:
rejoined['so2_cost_LOW_3pct']= rejoined['so2_ton']*rejoined['so2_cost_per_ton_LOW_3pct']
rejoined['so2_cost_HIGH_3pct']= rejoined['so2_ton']*rejoined['so2_cost_per_ton_HIGH_3pct']
rejoined['so2_cost_LOW_6pct']= rejoined['so2_ton']*rejoined['so2_cost_per_ton_LOW_6pct']
rejoined['so2_cost_HIGH_6pct']= rejoined['so2_ton']*rejoined['so2_cost_per_ton_HIGH_6pct']

rejoined['nox_cost_LOW_3pct']= rejoined['nox_ton']*rejoined['nox_cost_per_ton_LOW_3pct']
rejoined['nox_cost_HIGH_3pct']= rejoined['nox_ton']*rejoined['nox_cost_per_ton_HIGH_3pct']
rejoined['nox_cost_LOW_6pct']= rejoined['nox_ton']*rejoined['nox_cost_per_ton_LOW_6pct']
rejoined['nox_cost_HIGH_6pct']= rejoined['nox_ton']*rejoined['nox_cost_per_ton_HIGH_6pct']

In [36]:
summed_costs = rejoined.groupby(['date','hour']).sum().reset_index()

In [37]:
summed_costs['so2_cost_per_mwh_LOW_3pct'] = summed_costs['so2_cost_LOW_3pct']/summed_costs['load_MW']*(-1)
summed_costs['so2_cost_per_mwh_HIGH_3pct'] = summed_costs['so2_cost_HIGH_3pct']/summed_costs['load_MW']*(-1)
summed_costs['nox_cost_per_mwh_LOW_3pct'] = summed_costs['so2_cost_LOW_3pct']/summed_costs['load_MW']*(-1)
summed_costs['nox_cost_per_mwh_HIGH_3pct'] = summed_costs['so2_cost_HIGH_3pct']/summed_costs['load_MW']*(-1)

In [38]:
summed_costs['so2_cost_per_mwh_LOW_6pct'] = summed_costs['so2_cost_LOW_6pct']/summed_costs['load_MW']*(-1)
summed_costs['so2_cost_per_mwh_HIGH_6pct'] = summed_costs['so2_cost_HIGH_6pct']/summed_costs['load_MW']*(-1)
summed_costs['nox_cost_per_mwh_LOW_6pct'] = summed_costs['so2_cost_LOW_6pct']/summed_costs['load_MW']*(-1)
summed_costs['nox_cost_per_mwh_HIGH_6pct'] = summed_costs['so2_cost_HIGH_6pct']/summed_costs['load_MW']*(-1)

In [41]:
summed_costs['co2_short_tons_per_mwh'] = summed_costs['co2_short_tons']/summed_costs['load_MW']

In [42]:
summed_costs = summed_costs[['date','hour','co2_short_tons_per_mwh','so2_cost_per_mwh_LOW_3pct',
       'so2_cost_per_mwh_HIGH_3pct', 'nox_cost_per_mwh_LOW_3pct',
       'nox_cost_per_mwh_HIGH_3pct', 'so2_cost_per_mwh_LOW_6pct',
       'so2_cost_per_mwh_HIGH_6pct', 'nox_cost_per_mwh_LOW_6pct',
       'nox_cost_per_mwh_HIGH_6pct']]

In [43]:
summed_costs

,date,hour,co2_short_tons_per_mwh,so2_cost_per_mwh_LOW_3pct,so2_cost_per_mwh_HIGH_3pct,nox_cost_per_mwh_LOW_3pct,nox_cost_per_mwh_HIGH_3pct,so2_cost_per_mwh_LOW_6pct,so2_cost_per_mwh_HIGH_6pct,nox_cost_per_mwh_LOW_6pct,nox_cost_per_mwh_HIGH_6pct
0,2019-01-01,0,0.835887,4.562352,10.278854,4.562352,10.278854,4.179184,9.408701,4.179184,9.408701
1,2019-01-01,1,0.852033,4.722084,10.639123,4.722084,10.639123,4.325496,9.738474,4.325496,9.738474
2,2019-01-01,2,0.857347,4.719439,10.633296,4.719439,10.633296,4.323072,9.733141,4.323072,9.733141
3,2019-01-01,3,0.865476,4.481439,10.097178,4.481439,10.097178,4.105058,9.242409,4.105058,9.242409
4,2019-01-01,4,0.867741,4.273772,9.629224,4.273772,9.629224,3.914834,8.814069,3.914834,8.814069
...,...,...,...,...,...,...,...,...,...,...,...
8755,2019-12-31,19,0.598946,1.779735,4.007454,1.779735,4.007454,1.624932,3.656130,1.624932,3.656130
8756,2019-12-31,20,0.609769,1.745264,3.929656,1.745264,3.929656,1.593978,3.586314,1.593978,3.586314
8757,2019-12-31,21,0.616487,1.769799,3.984851,1.769799,3.984851,1.616932,3.637911,1.616932,3.637911
8758,2019-12-31,22,0.621569,2.071675,4.664998,2.071675,4.664998,1.894911,4.263757,1.894911,4.263757


In [44]:
summed_costs.to_csv('data/grid_emissions/ladwp_hourly_grid_noxSO2co2.csv')

In [154]:
#summed_emissions['ladwp_gas_gen_scaling_factor'] = summed_emissions['Natural_gas_Generation_(MWh)']/summed_emissions['load_MW_gas']

In [155]:
# summed_emissions['ladwp_gas_so2_lb'] = summed_emissions['so2_lb_gas']* summed_emissions['ladwp_gas_gen_scaling_factor']
# summed_emissions['ladwp_gas_nox_lb'] = summed_emissions['nox_lb_gas']* summed_emissions['ladwp_gas_gen_scaling_factor']
# summed_emissions['ladwp_gas_co2_short_tons'] = summed_emissions['co2_short_tons_gas']* summed_emissions['ladwp_gas_gen_scaling_factor']

In [156]:
# summed_emissions['ladwp_so2_lb'] = summed_emissions['ladwp_coal_so2_lb'] + summed_emissions['ladwp_gas_so2_lb']
# summed_emissions['ladwp_nox_lb'] = summed_emissions['ladwp_coal_nox_lb'] + summed_emissions['ladwp_gas_nox_lb']
# summed_emissions['ladwp_co2_short_tons'] = summed_emissions['ladwp_coal_co2_short_tons'] + summed_emissions['ladwp_gas_co2_short_tons']

In [157]:
# summed_emissions= summed_emissions[['date', 'hour', 'Total_Generation_(MWh)',
#        'ladwp_so2_lb', 'ladwp_nox_lb', 'ladwp_co2_short_tons']]

In [158]:
# summed_emissions['ladwp_so2_avg_lbs_per_mwh'] = summed_emissions['ladwp_so2_lb']/summed_emissions['Total_Generation_(MWh)']
# summed_emissions['ladwp_nox_avg_lbs_per_mwh'] = summed_emissions['ladwp_nox_lb']/summed_emissions['Total_Generation_(MWh)']
# summed_emissions['ladwp_co2_avg_short_tons_per_mwh'] = summed_emissions['ladwp_co2_short_tons']/summed_emissions['Total_Generation_(MWh)']

<ipython-input-158-d0a51e9bbeb1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  summed_emissions['ladwp_so2_avg_lbs_per_mwh'] = summed_emissions['ladwp_so2_lb']/summed_emissions['Total_Generation_(MWh)']
<ipython-input-158-d0a51e9bbeb1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  summed_emissions['ladwp_nox_avg_lbs_per_mwh'] = summed_emissions['ladwp_nox_lb']/summed_emissions['Total_Generation_(MWh)']
<ipython-input-158-d0a51e9bbeb1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy o